In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split

import re

## Importing Data

In [2]:
df = pd.read_csv("../data/train.csv")

In [3]:
df.head(4)

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1


## Column Description

UniqueID : Identifier for customers

loan_default:Payment default in the first EMI on due date

disbursed_amount:Amount of Loan disbursed

asset_cost:Cost of the Asset

ltv:Loan to Value of the asset

branch_id:Branch where the loan was disbursed

supplier_id:Vehicle Dealer where the loan was disbursed

manufacturer_id:Vehicle manufacturer(Hero, Honda, TVS etc.)

Current_pincode:Current pincode of the customer

Date.of.Birth:Date of birth of the customer

Employment.Type:Employment Type of the customer (Salaried/Self Employed)

DisbursalDate:Date of disbursement

State_ID:State of disbursement

Employee_code_ID:Employee of the organization who logged the disbursement

MobileNo_Avl_Flag:if Mobile no. was shared by the customer then flagged as 1

Aadhar_flag:if aadhar was shared by the customer then flagged as 1

PAN_flag:if pan was shared by the customer then flagged as 1

VoterID_flag:if voter was shared by the customer then flagged as 1

Driving_flag:if DL was shared by the customer then flagged as 1

Passport_flag:if passport was shared by the customer then flagged as 1

PERFORM_CNS.SCORE:Bureau Score

PERFORM_CNS.SCORE.DESCRIPTION:Bureau score description

PRI.NO.OF.ACCTS:count of total loans taken by the customer at the time of disbursement
Primary accounts are those which the customer has taken for his personal use

PRI.ACTIVE.ACCTS:count of active loans taken by the customer at the time of disbursement

PRI.OVERDUE.ACCTS:count of default accounts at the time of disbursement

PRI.CURRENT.BALANCE:total Principal outstanding amount of the active loans at the time of disbursement

PRI.SANCTIONED.AMOUNT:total amount that was sanctioned for all the loans at the time of disbursement

PRI.DISBURSED.AMOUNT:total amount that was disbursed for all the loans at the time of disbursement

SEC.NO.OF.ACCTS:count of total loans taken by the customer at the time of disbursement
Secondary accounts are those which the customer act as a co-applicant or gaurantor

SEC.ACTIVE.ACCTS:count of active loans taken by the customer at the time of disbursement

SEC.OVERDUE.ACCTS:count of default accounts at the time of disbursement

SEC.CURRENT.BALANCE:total Principal outstanding amount of the active loans at the time of disbursement

SEC.SANCTIONED.AMOUNT:total amount that was sanctioned for all the loans at the time of disbursement

SEC.DISBURSED.AMOUNT:total amount that was disbursed for all the loans at the time of disbursement

PRIMARY.INSTAL.AMT:EMI Amount of the primary loan

SEC.INSTAL.AMT:EMI Amount of the secondary loan

NEW.ACCTS.IN.LAST.SIX.MONTHS:New loans taken by the customer in last 6 months before the disbursment

DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS:Loans defaulted in the last 6 months

AVERAGE.ACCT.AGE:Average loan tenure

CREDIT.HISTORY:LENGTH:Time since first loan

NO.OF_INQUIRIES:Enquries done by the customer for loans

## EDA

In [4]:
print(f"Nos of rows: {df.shape[0]}, Nos of columns: {df.shape[1]}")

Nos of rows: 233154, Nos of columns: 41


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             233154 non-null  int64  
 1   disbursed_amount                     233154 non-null  int64  
 2   asset_cost                           233154 non-null  int64  
 3   ltv                                  233154 non-null  float64
 4   branch_id                            233154 non-null  int64  
 5   supplier_id                          233154 non-null  int64  
 6   manufacturer_id                      233154 non-null  int64  
 7   Current_pincode_ID                   233154 non-null  int64  
 8   Date.of.Birth                        233154 non-null  object 
 9   Employment.Type                      225493 non-null  object 
 10  DisbursalDate                        233154 non-null  object 
 11  State_ID     

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
UniqueID,233154.0,535917.573376,6.831569e+04,417428.00,476786.25,535978.5,595039.75,6.710840e+05
disbursed_amount,233154.0,54356.993528,1.297131e+04,13320.00,47145.00,53803.0,60413.00,9.905720e+05
asset_cost,233154.0,75865.068144,1.894478e+04,37000.00,65717.00,70946.0,79201.75,1.628992e+06
ltv,233154.0,74.746530,1.145664e+01,10.03,68.88,76.8,83.67,9.500000e+01
branch_id,233154.0,72.936094,6.983499e+01,1.00,14.00,61.0,130.00,2.610000e+02
supplier_id,233154.0,19638.635035,3.491950e+03,10524.00,16535.00,20333.0,23000.00,2.480300e+04
manufacturer_id,233154.0,69.028054,2.214130e+01,45.00,48.00,86.0,86.00,1.560000e+02
Current_pincode_ID,233154.0,3396.880247,2.238148e+03,1.00,1511.00,2970.0,5677.00,7.345000e+03
State_ID,233154.0,7.262243,4.482230e+00,1.00,4.00,6.0,10.00,2.200000e+01
Employee_code_ID,233154.0,1549.477148,9.752613e+02,1.00,713.00,1451.0,2362.00,3.795000e+03


## Segregate into Numeric and Categorical Columns

In [7]:
def separate_features(df, categorical_threshold=10):
    """
    function to return list of numeric and categorical columns from dataframe
    based on dtype as well as nos of unique values in a column.
    Eg: State ID or flag columns
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    # Identify categorical columns (including those disguised as numeric)
    categorical_cols = []
    extra_cols = []
    
    for col in df.columns:
        if col in numeric_cols:
            if df[col].nunique() <= categorical_threshold:
                categorical_cols.append(col)
        else:
            # For non-numeric columns, check cardinality
            if df[col].nunique() > categorical_threshold:
                extra_cols.append(col)
            else:
                categorical_cols.append(col)
    
    # Create DataFrames
    numeric_cols = list(set(numeric_cols) - set(categorical_cols) - set(extra_cols))
        
    return numeric_cols, categorical_cols, extra_cols

In [8]:
numeric_cols, categorical_cols, extra_cols = separate_features(df,categorical_threshold = 25)

In [9]:
print("numerical columns: ",numeric_cols)
print('-'*100)
print("Categorical columns: ",categorical_cols)
print('-'*100)
print("Extra columns: ",extra_cols)

numerical columns:  ['PRI.CURRENT.BALANCE', 'SEC.CURRENT.BALANCE', 'UniqueID', 'SEC.SANCTIONED.AMOUNT', 'supplier_id', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'asset_cost', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'SEC.DISBURSED.AMOUNT', 'Employee_code_ID', 'PRIMARY.INSTAL.AMT', 'ltv', 'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'Current_pincode_ID', 'branch_id', 'SEC.INSTAL.AMT', 'disbursed_amount', 'PERFORM_CNS.SCORE']
----------------------------------------------------------------------------------------------------
Categorical columns:  ['manufacturer_id', 'Employment.Type', 'State_ID', 'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.OVERDUE.ACCTS', 'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES', 'loan_default']
----------------------------------------------------------------------------------------------------
Extra columns:  ['Da

## Target Column

In [10]:
df['loan_default'].value_counts(normalize=True)*100

loan_default
0    78.292888
1    21.707112
Name: proportion, dtype: float64

#### So, we can say that it is an imbalanced dataset

## Bivariate Analysis of Categorical Features with respect to Target

In [11]:
def bivariate_analysis(df, feature, target):
    """
    Perform bivariate analysis of a categorical feature with respect to a binary target.
    
    Parameters:
    df (pandas.DataFrame): The dataframe containing the data.
    feature (str): The name of the categorical feature column.
    target (str): The name of the binary target column.
    
    Returns:
    tuple: (normalized_crosstab, cramers_v)
        - normalized_crosstab: A pandas DataFrame with the normalized cross-tabulation.
        - cramers_v: The Cramer's V statistic for strength of association.
    """
    
    # Create the cross-tabulation
    crosstab = pd.crosstab(df[feature], df[target], normalize='index').sort_values(by = 1,ascending = False)
    
    # Calculate Cramer's V
    confusion_matrix = pd.crosstab(df[feature], df[target])
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    min_dim = min(confusion_matrix.shape) - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim))
    
    return crosstab, cramers_v

In [12]:
result, association = bivariate_analysis(df, 'Employment.Type', 'loan_default')

print("Normalized Cross-tabulation:")
print(result)
print(f"\nCramer's V: {association:.3f}")

Normalized Cross-tabulation:
loan_default            0         1
Employment.Type                    
Self employed    0.772343  0.227657
Salaried         0.796542  0.203458

Cramer's V: 0.029


In [13]:
def overall_bivariate_analysis(df,cat_col_list,target_col):
    
    d = {}
    for col in cat_col_list:
        if col != target_col:
            result, association = bivariate_analysis(df, col, target_col)
            print(result)
            print('-'*50)
            d[col] = np.round(association,3)
    
    print("Cramer's V Strength of Association")
    print(pd.Series(d).sort_values())

In [14]:
overall_bivariate_analysis(df,cat_col_list = categorical_cols,target_col = 'loan_default')

loan_default            0         1
manufacturer_id                    
153              0.666667  0.333333
48               0.727469  0.272531
45               0.771501  0.228499
120              0.779250  0.220750
49               0.781213  0.218787
67               0.782536  0.217464
51               0.792053  0.207947
86               0.795406  0.204594
145              0.799486  0.200514
152              1.000000  0.000000
156              1.000000  0.000000
--------------------------------------------------
loan_default            0         1
Employment.Type                    
Self employed    0.772343  0.227657
Salaried         0.796542  0.203458
--------------------------------------------------
loan_default         0         1
State_ID                        
13            0.693413  0.306587
14            0.724134  0.275866
2             0.728606  0.271394
12            0.734442  0.265558
17            0.754197  0.245803
8             0.770515  0.229485
18            0.779933

C:\Users\abhid\AppData\Local\Temp\ipykernel_6224\2963784454.py:24: RuntimeWarning: invalid value encountered in divide
  cramers_v = np.sqrt(chi2 / (n * min_dim))


loan_default                                               0         1
PERFORM_CNS.SCORE.DESCRIPTION                                         
M-Very High Risk                                    0.695419  0.304581
L-Very High Risk                                    0.719577  0.280423
K-High Risk                                         0.721880  0.278120
I-Medium Risk                                       0.727371  0.272629
Not Scored: Sufficient History Not Available        0.744223  0.255777
J-High Risk                                         0.747599  0.252401
H-Medium Risk                                       0.758133  0.241867
No Bureau History Available                         0.768687  0.231313
Not Scored: Only a Guarantor                        0.786885  0.213115
Not Scored: Not Enough Info available on the cu...  0.790305  0.209695
G-Low Risk                                          0.802909  0.197091
Not Scored: No Updates available in last 36 months  0.809648  0.190352
F-Low 

## Feature Engg of Datetime columns --> DOB and Disbursal date

In [15]:
# Direct vectorized conversion (faster for large datasets)
df['Date.of.Birth'] = pd.to_datetime(df['Date.of.Birth'], dayfirst=True, errors='coerce')
df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'], dayfirst=True, errors='coerce')

C:\Users\abhid\AppData\Local\Temp\ipykernel_6224\3702760870.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date.of.Birth'] = pd.to_datetime(df['Date.of.Birth'], dayfirst=True, errors='coerce')
C:\Users\abhid\AppData\Local\Temp\ipykernel_6224\3702760870.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'], dayfirst=True, errors='coerce')


In [16]:
df['Date.of.Birth']

0        1984-01-01
1        1985-07-31
2        1985-08-24
3        1993-12-30
4        1977-12-09
            ...    
233149   1988-08-01
233150   1988-12-05
233151   1976-06-01
233152   1994-03-26
233153   1984-02-18
Name: Date.of.Birth, Length: 233154, dtype: datetime64[ns]

In [17]:
df['BirthMonth'] = df['Date.of.Birth'].dt.strftime("%Y-%m")
df['DisbursalMonth'] = df['DisbursalDate'].dt.strftime("%Y-%m")

### Bivariate Analysis of Disbursal Month

In [18]:
result, association = bivariate_analysis(df,'DisbursalMonth','loan_default')

print("Normalized Cross-tabulation:")
print(result)
print(f"\nCramer's V: {association:.3f}")

Normalized Cross-tabulation:
loan_default           0         1
DisbursalMonth                    
2018-10         0.764853  0.235147
2018-08         0.784330  0.215670
2018-09         0.808124  0.191876

Cramer's V: 0.043


### Age Column

In [19]:
df['age'] = np.round((df['DisbursalDate'] - df['Date.of.Birth']).dt.days/365.25)

In [20]:
df['age'] 

0         35.0
1         33.0
2         33.0
3         25.0
4         41.0
          ... 
233149    30.0
233150    30.0
233151    42.0
233152    25.0
233153    35.0
Name: age, Length: 233154, dtype: float64

### Create Avg Account Age and Credit history length to months

In [21]:
def convert_to_months(duration_str):
    # Use regular expressions to extract the years and months
    years_match = re.search(r'(\d+)\s*yrs?', duration_str)
    months_match = re.search(r'(\d+)\s*mon', duration_str)
    
    # Initialize years and months
    years = int(years_match.group(1)) if years_match else 0
    months = int(months_match.group(1)) if months_match else 0
    
    # Convert years to months and sum them
    total_months = (years * 12) + months
    return total_months

In [22]:
df['AVERAGE.ACCT.AGE'] = df['AVERAGE.ACCT.AGE'].apply(convert_to_months)
df['CREDIT.HISTORY.LENGTH'] = df['CREDIT.HISTORY.LENGTH'].apply(convert_to_months)

In [23]:
df['CREDIT.HISTORY.LENGTH']

0          0
1         23
2          0
3         15
4          0
          ..
233149    39
233150     6
233151     0
233152     0
233153     0
Name: CREDIT.HISTORY.LENGTH, Length: 233154, dtype: int64

## Binning of Numeric Variables

In [24]:
df.head(3)

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,BirthMonth,DisbursalMonth,age
0,420825,50578,58400,89.55,67,22807,45,1441,1984-01-01,Salaried,2018-08-03,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1984-01,2018-08,35.0
1,537409,47145,65550,73.23,67,22807,45,1502,1985-07-31,Self employed,2018-09-26,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,23,23,0,1,1985-07,2018-09,33.0
2,417566,53278,61360,89.63,67,22807,45,1497,1985-08-24,Self employed,2018-08-01,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1985-08,2018-08,33.0


In [25]:
numeric_cols = separate_features(df,categorical_threshold=5)[0]
numeric_cols = [col for col in numeric_cols if 'id' not in col.lower()]

In [26]:
numeric_cols

['PRI.OVERDUE.ACCTS',
 'PRI.CURRENT.BALANCE',
 'SEC.CURRENT.BALANCE',
 'SEC.SANCTIONED.AMOUNT',
 'NO.OF_INQUIRIES',
 'PRI.SANCTIONED.AMOUNT',
 'PRI.DISBURSED.AMOUNT',
 'SEC.NO.OF.ACCTS',
 'asset_cost',
 'PRI.NO.OF.ACCTS',
 'AVERAGE.ACCT.AGE',
 'PRI.ACTIVE.ACCTS',
 'SEC.DISBURSED.AMOUNT',
 'CREDIT.HISTORY.LENGTH',
 'PRIMARY.INSTAL.AMT',
 'ltv',
 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
 'age',
 'SEC.OVERDUE.ACCTS',
 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
 'SEC.ACTIVE.ACCTS',
 'SEC.INSTAL.AMT',
 'disbursed_amount',
 'PERFORM_CNS.SCORE']

In [27]:
df[numeric_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
PRI.OVERDUE.ACCTS,233154.0,0.156549,5.487867e-01,0.00,0.00,0.0,0.00,2.500000e+01
PRI.CURRENT.BALANCE,233154.0,165900.076936,9.422736e+05,-6678296.00,0.00,0.0,35006.50,9.652492e+07
SEC.CURRENT.BALANCE,233154.0,5427.792819,1.702370e+05,-574647.00,0.00,0.0,0.00,3.603285e+07
SEC.SANCTIONED.AMOUNT,233154.0,7295.923347,1.831560e+05,0.00,0.00,0.0,0.00,3.000000e+07
NO.OF_INQUIRIES,233154.0,0.206615,7.064977e-01,0.00,0.00,0.0,0.00,3.600000e+01
PRI.SANCTIONED.AMOUNT,233154.0,218503.855323,2.374794e+06,0.00,0.00,0.0,62500.00,1.000000e+09
PRI.DISBURSED.AMOUNT,233154.0,218065.898655,2.377744e+06,0.00,0.00,0.0,60800.00,1.000000e+09
SEC.NO.OF.ACCTS,233154.0,0.059081,6.267946e-01,0.00,0.00,0.0,0.00,5.200000e+01
asset_cost,233154.0,75865.068144,1.894478e+04,37000.00,65717.00,70946.0,79201.75,1.628992e+06
PRI.NO.OF.ACCTS,233154.0,2.440636,5.217233e+00,0.00,0.00,0.0,3.00,4.530000e+02


In [28]:
df['age'] = np.where(df['age']<0,df['age']+100,df['age'])

In [29]:
df['age'].describe()

count    233154.000000
mean         34.510907
std           9.834508
min          18.000000
25%          26.000000
50%          33.000000
75%          41.000000
max          69.000000
Name: age, dtype: float64

In [30]:
for col in numeric_cols:
    df[col] = df[col].astype('float64')
    num_bins = 5
    
    # Create a mask for values <= 0 and > 0,
    # since there are too many values<=0, this group treated separately
    # values greater than 0 are equal frequency binned
    mask_special = df[col] <= 0.0
    mask_positive = df[col] > 0.0
    
    # Initialize the binned column with NaN
    df[f'{col}_binned'] = np.nan
    
    # Assign 'Special' bin to values <= 0
    df.loc[mask_special, f'{col}_binned'] = 'Special (<= 0)'
    
    # Apply equal frequency binning to positive values
    df.loc[mask_positive, f'{col}_binned'] = pd.qcut(df.loc[mask_positive, col], q=num_bins, duplicates='drop')

C:\Users\abhid\AppData\Local\Temp\ipykernel_6224\3012144636.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Special (<= 0)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask_special, f'{col}_binned'] = 'Special (<= 0)'
C:\Users\abhid\AppData\Local\Temp\ipykernel_6224\3012144636.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Special (<= 0)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask_special, f'{col}_binned'] = 'Special (<= 0)'
C:\Users\abhid\AppData\Local\Temp\ipykernel_6224\3012144636.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Special (<= 0)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mas

In [31]:
df

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,BirthMonth,DisbursalMonth,age,PRI.OVERDUE.ACCTS_binned,PRI.CURRENT.BALANCE_binned,SEC.CURRENT.BALANCE_binned,SEC.SANCTIONED.AMOUNT_binned,NO.OF_INQUIRIES_binned,PRI.SANCTIONED.AMOUNT_binned,PRI.DISBURSED.AMOUNT_binned,SEC.NO.OF.ACCTS_binned,asset_cost_binned,PRI.NO.OF.ACCTS_binned,AVERAGE.ACCT.AGE_binned,PRI.ACTIVE.ACCTS_binned,SEC.DISBURSED.AMOUNT_binned,CREDIT.HISTORY.LENGTH_binned,PRIMARY.INSTAL.AMT_binned,ltv_binned,NEW.ACCTS.IN.LAST.SIX.MONTHS_binned,age_binned,SEC.OVERDUE.ACCTS_binned,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS_binned,SEC.ACTIVE.ACCTS_binned,SEC.INSTAL.AMT_binned,disbursed_amount_binned,PERFORM_CNS.SCORE_binned
0,420825,50578.0,58400.0,89.55,67,22807,45,1441,1984-01-01,Salaried,2018-08-03,6,1998,1,1,0,0,0,0,0.0,No Bureau History Available,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1984-01,2018-08,35.0,Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(36999.999, 64500.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(84.74, 95.0]",Special (<= 0),"(30.0, 36.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(45343.6, 51303.0]",Special (<= 0)
1,537409,47145.0,65550.0,73.23,67,22807,45,1502,1985-07-31,Self employed,2018-09-26,6,1998,1,1,0,0,0,0,598.0,I-Medium Risk,1.0,1.0,1.0,27600.0,50200.0,50200.0,0.0,0.0,0.0,0.0,0.0,0.0,1991.0,0.0,0.0,1.0,23.0,23.0,0.0,1,1985-07,2018-09,33.0,"(0.999, 2.0]","(13392.8, 42563.2]",Special (<= 0),Special (<= 0),Special (<= 0),"(30000.0, 66000.0]","(30000.0, 64942.4]",Special (<= 0),"(64500.0, 68764.0]","(0.999, 2.0]","(16.0, 26.0]","(0.999, 2.0]",Special (<= 0),"(19.0, 28.0]","(1939.0, 3786.0]","(66.35, 74.24]",Special (<= 0),"(30.0, 36.0]",Special (<= 0),"(0.999, 20.0]",Special (<= 0),Special (<= 0),"(45343.6, 51303.0]","(368.0, 628.0]"
2,417566,53278.0,61360.0,89.63,67,22807,45,1497,1985-08-24,Self employed,2018-08-01,6,1998,1,1,0,0,0,0,0.0,No Bureau History Available,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1985-08,2018-08,33.0,Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(36999.999, 64500.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(84.74, 95.0]",Special (<= 0),"(30.0, 36.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(51303.0, 56195.6]",Special (<= 0)
3,624493,57513.0,66113.0,88.48,67,22807,45,1501,1993-12-30,Self employed,2018-10-26,6,1998,1,1,0,0,0,0,305.0,L-Very High Risk,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,8.0,15.0,1.0,1,1993-12,2018-10,25.0,Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(0.999, 2.0]",Special (<= 0),Special (<= 0),Special (<= 0),"(64500.0, 68764.0]","(2.0, 4.0]","(6.0, 11.0]",Special (<= 0),Special (<= 0),"(9.0, 19.0]","(0.999, 1939.0]","(84.74, 95.0]",Special (<= 0),"(17.999, 25.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(56195.6, 62447.0]","(10.999, 368.0]"
4,539055,52378.0,60300.0,88.39,67,22807,45,1495,1977-12-09,Self employed,2018-09-26,6,1998,1,1,0,0,0,0,0.0,No Bureau History Available,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1977-12

### Bivariate Analysis on Binned Columns

In [32]:
binned_cols = [col for col in df.columns if 'binned' in col]

In [33]:
len(binned_cols)

24

In [34]:
overall_bivariate_analysis(df,cat_col_list = binned_cols,target_col = 'loan_default')

loan_default                     0         1
PRI.OVERDUE.ACCTS_binned                    
(2.0, 25.0]               0.710434  0.289566
(0.999, 2.0]              0.727134  0.272866
Special (<= 0)            0.790177  0.209823
--------------------------------------------------
loan_default                       0         1
PRI.CURRENT.BALANCE_binned                    
(42563.2, 116030.4]         0.762773  0.237227
(13392.8, 42563.2]          0.770135  0.229865
Special (<= 0)              0.774560  0.225440
(0.999, 13392.8]            0.790078  0.209922
(116030.4, 449238.8]        0.813317  0.186683
(449238.8, 96524920.0]      0.843698  0.156302
--------------------------------------------------
loan_default                       0         1
SEC.CURRENT.BALANCE_binned                    
(35203.6, 88202.8]          0.770045  0.229955
(9727.6, 35203.6]           0.781818  0.218182
Special (<= 0)              0.782529  0.217471
(0.999, 9727.6]             0.815431  0.184569
(88202.8, 35780

## Dividing Variables into Dev, OOS and OOT

In [35]:
df['DisbursalMonth'].value_counts()

DisbursalMonth
2018-10    98364
2018-08    68002
2018-09    66788
Name: count, dtype: int64

In [36]:
df['set'] = np.where(df['DisbursalMonth'] == '2018-10','oot','dev_oos')

In [40]:
# splitting dev-oos into 80/20 ratio
dev, oos = train_test_split(df[df["set"]=="dev_oos"].reset_index(), test_size=0.2)

In [42]:
dev["set"] ="dev"
oos["set"]= "oos"
oot = df[df["set"]=="oot"]
df = pd.concat([dev,oos,oot])

In [43]:
df.head(3)

,index,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,BirthMonth,DisbursalMonth,age,PRI.OVERDUE.ACCTS_binned,PRI.CURRENT.BALANCE_binned,SEC.CURRENT.BALANCE_binned,SEC.SANCTIONED.AMOUNT_binned,NO.OF_INQUIRIES_binned,PRI.SANCTIONED.AMOUNT_binned,PRI.DISBURSED.AMOUNT_binned,SEC.NO.OF.ACCTS_binned,asset_cost_binned,PRI.NO.OF.ACCTS_binned,AVERAGE.ACCT.AGE_binned,PRI.ACTIVE.ACCTS_binned,SEC.DISBURSED.AMOUNT_binned,CREDIT.HISTORY.LENGTH_binned,PRIMARY.INSTAL.AMT_binned,ltv_binned,NEW.ACCTS.IN.LAST.SIX.MONTHS_binned,age_binned,SEC.OVERDUE.ACCTS_binned,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS_binned,SEC.ACTIVE.ACCTS_binned,SEC.INSTAL.AMT_binned,disbursed_amount_binned,PERFORM_CNS.SCORE_binned,set
9003,16010.0,485964,43394.0,63711.0,70.63,18,13968,45,2695,1982-05-14,Salaried,2018-08-31,4,78,1,1,0,0,0,0,738.0,C-Very Low Risk,3.0,2.0,0.0,9641.0,12389.0,12389.0,0.0,0.0,0.0,0.0,0.0,0.0,950.0,0.0,2.0,0.0,3.0,7.0,0.0,0,1982-05,2018-08,36.0,Special (<= 0),"(0.999, 13392.8]",Special (<= 0),Special (<= 0),Special (<= 0),"(0.999, 30000.0]","(0.999, 30000.0]",Special (<= 0),"(36999.999, 64500.0]","(2.0, 4.0]","(0.999, 6.0]","(0.999, 2.0]",Special (<= 0),"(0.999, 9.0]","(0.999, 1939.0]","(66.35, 74.24]","(0.999, 2.0]","(30.0, 36.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(13319.999, 45343.6]","(709.0, 762.0]",dev
3545,6427.0,552282,54313.0,67100.0,83.31,29,22746,86,5898,1992-01-01,Self employed,2018-09-30,3,1622,1,1,0,0,0,0,0.0,No Bureau History Available,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1992-01,2018-09,27.0,Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(64500.0, 68764.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(79.46, 84.74]",Special (<= 0),"(25.0, 30.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(51303.0, 56195.6]",Special (<= 0),dev
78397,133549.0,524442,55559.0,73749.0,76.34,136,23982,86,3738,2071-06-01,Self employed,2018-09-21,8,1304,1,1,0,0,0,0,733.0,D-Very Low Risk,9.0,2.0,0.0,63971.0,150000.0,150000.0,0.0,0.0,0.0,0.0,0.0,0.0,93555.0,0.0,0.0,0.0,9.0,25.0,0.0,0,2071-06,2018-09,47.0,Special (<= 0),"(42563.2, 116030.4]",Special (<= 0),Special (<= 0),Special (<= 0),"(66000.0, 172163.0]","(64942.4, 170000.0]",Special (<= 0),"(73528.0, 83091.4]","(7.0, 453.0]","(6.0, 11.0]","(0.999, 2.0]",Special (<= 0),"(19.0, 28.0]","(19991.2, 25642806.0]","(74.24, 79.46]",Special (<= 0),"(44.0, 69.0]",Special (<= 0),Special (<= 0),Special (<= 0),Special (<= 0),"(51303.0, 56195.6]","(709.0, 762.0]",dev


### Dropping the learned params before model training

In [44]:
learned_cols = [col for col in df.columns if 'binned' in col] + ['BirthMonth','UniqueID','index','Current_pincode_ID','Employee_code_ID']

In [46]:
df.drop(columns=learned_cols,inplace=True)

## Save the Dev, OOS and OOT data sets

In [48]:
for t in ['dev','oos','oot']:
    df[df['set'] == t].to_csv(f'../data/{t}.csv',index=False)